In [3]:
import numpy as np
import pandas as pd
import tensorflow  as tf

In [4]:
tf.__version__

'2.13.0'

In [6]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


## Part 1- Data Preprocessing

In [9]:
dataset = pd.read_csv('drive/My Drive/Deep Learning A-Z/ANN/Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [10]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [11]:
print(y)

[1 0 1 ... 1 1 0]


label encoding "gender" column

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


one hot encoding "Geography" column

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer( transformers=[('encoder' , OneHotEncoder() , [1])] , remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [16]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


split dataset into training set and test set

In [17]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=0)

feature scaling

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

initialize the ANN

In [19]:
ann = tf.keras.models.Sequential()

adding the inner layer and the first hidden layer

In [20]:
ann.add(tf.keras.layers.Dense(units=6 , activation='relu'))

adding second hidden layer

In [21]:
ann.add(tf.keras.layers.Dense(units=6 , activation='relu'))

adding output layer

In [22]:
ann.add(tf.keras.layers.Dense(units=1 , activation='sigmoid'))

## Part 3 - Training the ANN

compile the ANN

In [23]:
ann.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=['accuracy'])

train ANN on training set

In [24]:
ann.fit(X_train , y_train , batch_size=32 , epochs=100)

Epoch 1/100
250/250 [==============================] - 3s 3ms/step - loss: 0.5981 - accuracy: 0.6881
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4727 - accuracy: 0.7929
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4407 - accuracy: 0.8006
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4236 - accuracy: 0.8090
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4089 - accuracy: 0.8164
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3951 - accuracy: 0.8275
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3840 - accuracy: 0.8350
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3743 - accuracy: 0.8432
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3671 - accuracy: 0.8482
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3611 - accura

## Part 4 - Making predictions and evaluating model

In [26]:
ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5

1/1 [==============================] - 0s 38ms/step


array([[False]])

predicting test set results

In [27]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1) , y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


making confusion matrix

In [28]:
from sklearn.metrics import confusion_matrix , accuracy_score
cm = confusion_matrix(y_test , y_pred)
print(cm)
accuracy_score(y_test , y_pred)

[[1517   78]
 [ 200  205]]


0.861